In [5]:
# utils imports
import pdb
import time
import numpy as np
import pandas as pd
import random
from itertools import count
from collections import deque, namedtuple
from copy import deepcopy as dcopy
import math
import os
import matplotlib.pyplot as plt

# PyTorch imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [6]:
class SimpleEnv():
    def __init__(self, n_inputs=2, n_outputs=2):
        self.n_inputs = n_inputs
        self.n_outputs = n_outputs
    
    def step(self, state, action):
        next_state = state.clone()
        
        if action[0] == 0:
            next_state[0][0] = state[0][0] - 1
        else:
            next_state[0][0] = state[0][0] + 1

        if next_state[0][0] == 1 and state[0][1] == 1:
            next_state[0][1] = 0

        reward = self.calc_reward(state, next_state)

        done = False
        if next_state[0][0] == 0 or next_state[0][0] == 3:
            done = True

        return next_state, reward, done

    def reset(self):
        state = torch.Tensor([[2,1]])
        return state

    def calc_reward(self, state, next_state):
        if next_state[0][0] == 0:
            return -10
        elif next_state[0][0] == 3:
            return 10
        if state[0][1] == 1 and next_state[0][1] == 0:
            return 3
        else:
            return 0

In [7]:
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 

env = SimpleEnv()

class ReplayMemory(object):

    def __init__(self, capacity):
        self.memory = deque([],maxlen=capacity)

    def push(self, *args):
        """Save a transition"""
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

In [8]:
class DQN(nn.Module):
    def __init__(self, n_inputs, n_outputs, n_neurons):
        super(DQN, self).__init__()
        self.lin1 = nn.Linear(n_inputs, n_neurons)
        self.lin2 = nn.Linear(n_neurons, n_neurons)
        self.lin3 = nn.Linear(n_neurons, n_outputs)
    
    def forward(self, x):
        x = F.relu(self.lin1(x))
        x = F.relu(self.lin2(x))
        return self.lin3(x)

    def get_action(state):
        global steps_done
        sample = random.random()
        eps_threshold = EPS_END + (EPS_START - EPS_END) * \
            math.exp(-1. * steps_done / EPS_DECAY)
        steps_done += 1
        if sample > eps_threshold:
            with torch.no_grad():
                # t.max(1) will return largest column value of each row.
                # second column on max result is index of where max element was
                # found, so we pick action with the larger expected reward.
                return policy_net(state).max(1)[1].view(1, 1)
        else:
            return torch.tensor([[random.randrange(env.n_outputs)]], device=device, dtype=torch.long)


In [9]:
BATCH_SIZE = 128
GAMMA = 0.999
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 200
TARGET_UPDATE = 10

# Get screen size so that we can initialize layers correctly based on shape
# returned from AI gym. Typical dimensions at this point are close to 3x40x90
# which is the result of a clamped and down-scaled render buffer in get_screen()
state = env.reset()

policy_net = DQN(env.n_inputs, env.n_outputs, 24).to(device)
target_net = DQN(env.n_inputs, env.n_outputs, 24).to(device)
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()

optimizer = optim.RMSprop(policy_net.parameters())
memory = ReplayMemory(10000)

episode_durations = []
steps_done = 0

In [13]:
def optimize_model():
    if len(memory) < BATCH_SIZE:
        return
    transitions = memory.sample(BATCH_SIZE)
    # Transpose the batch (see https://stackoverflow.com/a/19343/3343043 for
    # detailed explanation). This converts batch-array of Transitions
    # to Transition of batch-arrays.
    batch = Transition(*zip(*transitions))

    # Compute a mask of non-final states and concatenate the batch elements
    # (a final state would've been the one after which simulation ended)
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                          batch.next_state)), device=device, dtype=torch.bool)
    non_final_next_states = torch.cat([s for s in batch.next_state
                                                if s is not None])
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)

    # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
    # columns of actions taken. These are the actions which would've been taken
    # for each batch state according to policy_net
    state_action_values = policy_net(state_batch).gather(1, action_batch)

    # Compute V(s_{t+1}) for all next states.
    # Expected values of actions for non_final_next_states are computed based
    # on the "older" target_net; selecting their best reward with max(1)[0].
    # This is merged based on the mask, such that we'll have either the expected
    # state value or 0 in case the state was final.
    next_state_values = torch.zeros(BATCH_SIZE, device=device)
    next_state_values[non_final_mask] = target_net(non_final_next_states).max(1)[0].detach()
    # Compute the expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch

    # Compute Huber loss
    criterion = nn.SmoothL1Loss()
    loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1))

    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    # for param in policy_net.parameters():
    #     param.grad.data.clamp_(-1, 1)
    optimizer.step()

    return loss.item()

In [15]:
num_episodes = 1000
for i_episode in range(num_episodes):
    # Initialize the environment and state
    state = env.reset()
    for t in count():
        # Select and perform an action
        action = DQN.get_action(state)
        next_state, reward, done = env.step(state, action)
        reward = torch.tensor([reward], device=device)
        print(action.item(), reward.item())
        # Store the transition in memory
        memory.push(state, action, next_state, reward)

        # Move to the next state
        state = next_state

        # Perform one step of the optimization (on the policy network)
        loss = optimize_model()
        print(loss)
        if done:
            episode_durations.append(t + 1)
            # plot_durations()
            break
    # Update the target network, copying all weights and biases in DQN
    if i_episode % TARGET_UPDATE == 0:
        target_net.load_state_dict(policy_net.state_dict())

print('Complete')

1 10
5863117.5
1 10
6254291.5
1 10
6283945.0
1 10
6010394.5
0 3
6115341.0
1 0
5768244.5
1 10
6098571.5
1 10
6223347.5
1 10
5902441.0
1 10
6131478.0
0 3
6089261.5
1 0
5941923.0
1 10
5971991.0
1 10
5877986.5
1 10
5759935.5
1 10
6028001.0
1 10
6108084.0
1 10
5984286.0
1 10
5810755.5
1 10
6195615.0
1 10
6074069.0
1 10
5899933.0
1 10
6307286.5
1 10
6234277.0
1 10
6195415.0
1 10
6334524.5
0 3
6209659.0
1 0
6290325.0
1 10
6268667.0
1 10
6195418.5
1 10
6226028.0
1 10
6206960.0
1 10
6080427.5
1 10
6209932.5
1 10
6016818.5
1 10
6019593.5
1 10
5976977.5
1 10
6426575.5
1 10
6011667.0
1 10
6199687.0
1 10
6204110.0
1 10
6260335.5
1 10
6057986.0
1 10
6010001.0
1 10
6143989.0
1 10
5915669.5
1 10
6153505.5
1 10
6294146.0
1 10
6248640.0
1 10
6225442.0
1 10
6151625.0
1 10
6236450.0
1 10
6553981.0
1 10
6244114.0
1 10
5913252.5
1 10
6333719.5
1 10
6212232.5
1 10
6400450.5
1 10
6247010.0
1 10
6143803.5
1 10
6388647.5
1 10
6076385.0
1 10
6343117.5
1 10
6322157.0
1 10
6221652.0
1 10
6231098.0
1 10
6343995.5
1